In [3]:
# Challenge Sorbonne - DST
#
#!pip install torch_geometric

import numpy as np
import pandas as pd
import os
import time
import os
import json
#import rarfile
import networkx as nx
import io
import re
import random
import csv
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Subset
from torch.optim import AdamW
!pip install torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
import networkx as nx

from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tqdm as notebook_tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

#env_system = os.environ
#print(env_system)

device is cpu
The current working directory is: c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v12


In [4]:
# for Google Colab only
if 'COLAB_RELEASE_TAG' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')

In [5]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

# init du repertoire des données
if current_directory == '/content': # Google Colab
    training_path_dir =   '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_training_set'
    test_path_dir =       '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_test_set'
    train_meta_data =     '/content/drive/MyDrive/Sorbonne_Data_challenge/training_set_metadata.csv'

    model_save_file =     '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/model_sorbonne_weights.pth'
    file_split_training = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/split_val-colab-v12.csv'
    split_char = '/'

    test_init_file =        '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/test_set_to_predict.csv'
    filename_test_results = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/test_prediction.csv'
    filename_trained =      '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/list_hash_trained.csv'

elif current_directory == r"c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v12": #PC1
    split_char = '\\'

    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\Pred_test_v12_500_2803\list_hash_trained.csv"

    train_meta_data =  r'..\training_set_metadata.csv'
    file_split_training = r'C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\Pred_test_v13\split_val-pc-v13.csv'

    #test_path_dir =   r"D:\ChallengeDST\folder_training_set\folder_training_set"
    #test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    #test_path_dir = r"G:\Mon Drive\Colab Notebooks\Sorbonne-Challenge"
    #test_init_file = r".\test_set_to_predict.csv"
    #filename_test_results = r".\test_prediction.csv"

    model_save_file =  r'..\Pred_test_v11_15K_2503\model_sorbonne_weights.pth'
    #filename_test_results = r".\test_prediction.csv"

elif current_directory == r"c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne": #PC2
    split_char = '\\'

    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r".\list_hash_trained.csv"

    train_meta_data =  'training_set_metadata.csv'
    file_split_training = r'.\split_training-pc.csv'

    #test_path_dir =   r"D:\ChallengeDST\folder_training_set\folder_training_set"
    #test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    test_path_dir = r"G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_test_set"
    test_init_file = "test_set_to_predict.csv"
    filename_test_results = "test_prediction.csv"
    model_save_file =  'model_sorbonne_weights.pth'
else:
    print("**** ERROR: init files names and directories not done - root directory and environment not identified")


# read CSV input and save in df
df_meta_train = pd.read_csv(train_meta_data, sep=";")
df_meta_train.head()


The current working directory is: c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v12


,name,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,allocate memory,...,winzip,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda
0,9fbf213113ba0a18dc2642f83b1201541428fd7951d6a8...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1b35c9dbf3cd9ac60015aaa6cd451c898defa6dac1ff43...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,bf8d307a136a936f7338c1f2eec773c4eb1c802cab77da...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1e51933903f0358c0b635f863368eb15a61cd3442bc5bf...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8a6503fe68d699f8a31531c157e9da931192cd7e3ec809...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_meta_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23102 entries, 0 to 23101
Columns: 454 entries, name to yoda
dtypes: int64(453), object(1)
memory usage: 80.0+ MB


In [7]:
print(filename_trained)
df_files_trained = pd.read_csv(filename_trained, sep=",")
print("taille :",len(df_files_trained))
df_files_trained.head()


C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\Pred_test_v12_500_2803\list_hash_trained.csv
taille : 497


,files_trained
0,861f2c5f07c9e1c7d24c2e34eb47ff3129cd39a2227a25...
1,bd880010492d29b7ea382eeb76405e6251cf9f51aa8d17...
2,f993af4662324130ca817cf3c7b04d5701a85422cb773e...
3,370a7ee4dd61e0951a080e4d9d5a29b7b2bddfc832d838...
4,5fb5b616dc35dc898c5b823f3393838b25b57bfd92d078...


In [8]:
# load & check traing split
df_training_split = pd.read_csv(file_split_training, sep=",")
full_file_list = df_training_split[df_training_split['batch'] == 'rep_0']['orign path'].tolist()
print(df_training_split['batch'].unique())
print(full_file_list[:5])
print(df_training_split.info())


['rep_0']
['G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\40936a67037f8fd8e215b045f9cdf9c55840411316a62c85c8b54f75c6b0a5c8.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\368d2b5ccfd49f01942f462037710146ec3ca5ca8a5318a092bc49bfebfe8bad.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\75010f6d21b0b4451b5465ab8f46b385bb1edc15ed50634f4120352edc49cf3a.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\95e944577e686782e9d725e2e79ef9657a3f3b4f3b9cb265487bcd8e55cba95d.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\72356f26be98c580e23ca7baebe665c781fb2484575b9b05ad38676ad74ffb9e.json']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data

In [9]:
def process_graph_directory(file_directory, val_size=0.1, rep_batch='rep_0', max_file=0):
    """
    Scan the training directory
    get the hash name of the files, check and exclude hash file with error
    retour a list of hash for training, validation and error
    """

    list_train_hash, list_val_hash, list_err_hash  = [], [], []
    file_with_err = 0
    count_files = 0
    #max_file = 0

    #file_split_training = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\ChallengeSorbonne\split_training-colab.csv"
    df_files_trained = pd.read_csv(filename_trained, sep=",")
    df_training_split = pd.read_csv(file_split_training, sep=",")
    full_file_list = df_training_split[df_training_split['batch']== rep_batch]['orign path'].tolist()

    for full_path_file in full_file_list:
        # test if filename in the trained list
        hash_name = full_path_file.split('.jso')[0]
        hash_name = hash_name.split(split_char)[-1]

        # file already trained
        if hash_name in df_files_trained['files_trained'].values:
        #    print("trained")
            continue

        # files not in metadata
        if hash_name not in df_meta_train['name'].values:
            list_err_hash.append(full_path_file)
            print("meta", hash_name, full_path_file)
            continue

        if os.path.exists(full_path_file):
            file_size = os.path.getsize(full_path_file)
            if file_size/1000 > 200_000: #file too big
                list_err_hash.append(full_path_file)
                print("size", file_size/1000, full_path_file)
                continue
        else:
            list_err_hash.append(full_path_file)
            print("not exist", full_path_file)
            continue

        # check if nb of files is within the max_files allowed
        if max_file != 0 and count_files > max_file:
            list_err_hash.append(full_path_file)
            print("count")
            break

        list_val_hash.append(full_path_file)
        """
        if count_files % (val_size*100) == 0:
            list_val_hash.append(full_path_file)
        else:
            list_train_hash.append(full_path_file)

        count_files += 1
        """
    print(f"Number of files with error: {len(list_err_hash)}")
    return list_train_hash, list_val_hash, list_err_hash

In [10]:
list_train, list_val, list_err = process_graph_directory(training_path_dir, val_size=0.01, rep_batch='rep_0')
print(len(list_val), len(list_train), len(list_err))

Number of files with error: 0
81 0 0


In [11]:
class HierarchicalAssemblyTokenizer:
    def __init__(self):
        # Tokenizer implementation from previous code
        # Abbreviated for brevity but would be the full implementation
        self.node_type_vocab = {}
        self.operation_vocab = {}
        self.register_vocab = {}
        self.memory_pattern_vocab = {}
        self.immediate_vocab = {}
        self.UNK_TOKEN = "<UNK>"

        # Regex patterns for parsing
        self.patterns = {
            'node_type': re.compile(r'([A-Z]+)\s*:'),
            'operation': re.compile(r':\s*([a-z]+)'),
            'registers': re.compile(r'(?:^|\s+)([a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l)(?:$|\s+|,|\])'),
            'memory_ref': re.compile(r'\[(.*?)\]'),
            'immediate': re.compile(r'0x[0-9a-fA-F]+|\d+')
        }

    def fit(self, graph_l, min_freq=1):
        """Build vocabularies from the digraph nodes"""
        node_types = []
        operations = []
        registers = []
        memory_patterns = []
        immediates = []
        self.unique_hashes = []
        self.hash_to_id = {}
        self.id_to_hash = {}

        # Extract features from node labels
        for item in graph_l:
            graph = item['graph_input']
            for node_id, node_attr in graph.nodes(data=True):
                label = node_attr.get('label', '')

                # Extract node type (JCC, INST)
                node_type_match = self.patterns['node_type'].search(label)
                if node_type_match:
                    node_types.append(node_type_match.group(1))

                # Extract operation (xor, push, mov)
                op_match = self.patterns['operation'].search(label)
                if op_match:
                    operations.append(op_match.group(1))

                # Extract registers
                reg_matches = self.patterns['registers'].findall(label)
                registers.extend(reg_matches)

                # Extract memory reference patterns
                mem_matches = self.patterns['memory_ref'].findall(label)
                for mem in mem_matches:
                    # Convert memory reference to a pattern (e.g., "reg + offset")
                    pattern = self._memory_to_pattern(mem)
                    memory_patterns.append(pattern)

                # Extract immediate values
                imm_matches = self.patterns['immediate'].findall(label)
                immediates.extend(imm_matches)

            self.unique_hashes.append(item['name'])
            self.unique_hashes = sorted(list(self.unique_hashes))
            for i, hash_val in enumerate(self.unique_hashes):
                self.hash_to_id[hash_val] = i
                self.id_to_hash[i] = hash_val

        # Build vocabularies with frequency filtering
        self._build_vocab(self.node_type_vocab, node_types, min_freq)
        self._build_vocab(self.operation_vocab, operations, min_freq)
        self._build_vocab(self.register_vocab, registers, min_freq)
        self._build_vocab(self.memory_pattern_vocab, memory_patterns, min_freq)
        self._build_vocab(self.immediate_vocab, immediates, min_freq)

        return self

    def fit_from_counts(self, all_counts, min_freq=3):
        """Fits the tokenizer from accumulated counts."""
        self.node_type_vocab.clear()
        self.operation_vocab.clear()
        self.register_vocab.clear()
        self.memory_pattern_vocab.clear()
        self.immediate_vocab.clear()

        self._build_vocab(self.node_type_vocab, all_counts['node_types'], min_freq)
        self._build_vocab(self.operation_vocab, all_counts['operations'], min_freq*2)
        self._build_vocab(self.register_vocab, all_counts['registers'], min_freq)
        self._build_vocab(self.memory_pattern_vocab, all_counts['memory_patterns'], min_freq*3)
        self._build_vocab(self.immediate_vocab, all_counts['immediates'], min_freq*3)

        self.unique_hashes = all_counts['unique_hashes']
        self.unique_hashes = sorted(list(self.unique_hashes))
        for i, hash_val in enumerate(self.unique_hashes):
            self.hash_to_id[hash_val] = i
            self.id_to_hash[i] = hash_val

    def _build_vocab(self, vocab_dict, tokens, min_freq):
        """Build vocabulary with frequency filtering"""
        counter = Counter(tokens)
        # Add special UNK token
        vocab_dict[self.UNK_TOKEN] = 0

        # Add tokens that meet minimum frequency
        idx = 1
        for token, count in counter.most_common():
            if count >= min_freq:
                vocab_dict[token] = idx
                idx += 1

    def _memory_to_pattern(self, mem_ref):
        """Convert memory reference to pattern"""
        # Replace registers with REG placeholder
        pattern = re.sub(r'[a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l', 'REG', mem_ref)
        # Replace immediate values with IMM placeholder
        pattern = re.sub(r'0x[0-9a-fA-F]+|\d+', 'IMM', pattern)
        return pattern.strip()

    def tokenize(self, node_label):
        """Tokenize a node label into hierarchical features"""
        features = {
            'node_type': self.UNK_TOKEN,
            'operation': self.UNK_TOKEN,
            'registers': [],
            'memory_pattern': self.UNK_TOKEN,
            'immediate': self.UNK_TOKEN
        }

        # Extract node type
        node_type_match = self.patterns['node_type'].search(node_label)
        if node_type_match:
            node_type = node_type_match.group(1)
            features['node_type'] = node_type if node_type in self.node_type_vocab else self.UNK_TOKEN

        # Extract operation
        op_match = self.patterns['operation'].search(node_label)
        if op_match:
            operation = op_match.group(1)
            features['operation'] = operation if operation in self.operation_vocab else self.UNK_TOKEN

        # Extract registers
        reg_matches = self.patterns['registers'].findall(node_label)
        features['registers'] = [reg if reg in self.register_vocab else self.UNK_TOKEN for reg in reg_matches]

        # Extract memory reference
        mem_matches = self.patterns['memory_ref'].findall(node_label)
        if mem_matches:
            pattern = self._memory_to_pattern(mem_matches[0])
            features['memory_pattern'] = pattern if pattern in self.memory_pattern_vocab else self.UNK_TOKEN

        # Extract immediate values
        imm_matches = self.patterns['immediate'].findall(node_label)
        if imm_matches:
            imm = imm_matches[0]
            features['immediate'] = imm if imm in self.immediate_vocab else self.UNK_TOKEN

        return features

    def encode_nodelabel(self, node_label):
        """Convert a node label to numeric feature vectors"""
        features = self.tokenize(node_label)

        # Encode each feature
        node_type_idx = self.node_type_vocab.get(features['node_type'], self.node_type_vocab[self.UNK_TOKEN])
        operation_idx = self.operation_vocab.get(features['operation'], self.operation_vocab[self.UNK_TOKEN])

        # Encode registers (take up to 3, pad if fewer)
        register_indices = []
        for i in range(min(3, len(features['registers']))):
            reg = features['registers'][i]
            reg_idx = self.register_vocab.get(reg, self.register_vocab[self.UNK_TOKEN])
            register_indices.append(reg_idx)

        # Pad register indices if needed
        while len(register_indices) < 3:
            register_indices.append(0)  # 0 for padding

        memory_pattern_idx = self.memory_pattern_vocab.get(
            features['memory_pattern'],
            self.memory_pattern_vocab[self.UNK_TOKEN]
        )

        immediate_idx = self.immediate_vocab.get(
            features['immediate'],
            self.immediate_vocab[self.UNK_TOKEN]
        )

        # Combine all indices into a feature vector
        encoded = np.array([
            node_type_idx,
            operation_idx,
            register_indices[0],
            register_indices[1],
            register_indices[2],
            memory_pattern_idx,
            immediate_idx
        ], dtype=np.int64)

        return encoded

    def encode_graph(self, digraph):
        """Convert an entire digraph to node feature vectors"""
        node_features = {}

        for node_id in digraph.nodes():
            label = digraph.nodes[node_id].get('label', '')
            node_features[node_id] = self.encode_nodelabel(label)

        return node_features

    def encode_hash(self, hash_file):
        return self.hash_to_id.get(hash_file, -1) # Return -1 if hash is not found

    def get_vocab_sizes(self):
        """Return the size of each vocabulary"""
        return {
            'node_type': len(self.node_type_vocab),
            'operation': len(self.operation_vocab),
            'register': len(self.register_vocab),
            'memory_pattern': len(self.memory_pattern_vocab),
            'immediate': len(self.immediate_vocab)
        }

    def get_tokens_and_counts(self, graph_l):
        """Extracts tokens and counts from graph_l."""
        node_types = []
        operations = []
        registers = []
        memory_patterns = []
        immediates = []
        unique_hashes = []
        self.hash_to_id = {}
        self.id_to_hash = {}

        for item in graph_l:
            graph = item['graph_input']
            for node_id, node_attr in graph.nodes(data=True):
                label = node_attr.get('label', '')

                node_type_match = self.patterns['node_type'].search(label)
                if node_type_match:
                    node_types.append(node_type_match.group(1))

                op_match = self.patterns['operation'].search(label)
                if op_match:
                    operations.append(op_match.group(1))

                reg_matches = self.patterns['registers'].findall(label)
                registers.extend(reg_matches)

                mem_matches = self.patterns['memory_ref'].findall(label)
                for mem in mem_matches:
                    pattern = self._memory_to_pattern(mem)
                    memory_patterns.append(pattern)

                imm_matches = self.patterns['immediate'].findall(label)
                immediates.extend(imm_matches)

            unique_hashes.append(item['name'])

        return {
            'node_types': node_types,
            'operations': operations,
            'registers': registers,
            'memory_patterns': memory_patterns,
            'immediates': immediates,
            'unique_hashes': unique_hashes,
        }

class AssemblyGraphDataset:
    def __init__(self, graph_list, tokenizer, node_feature_dim=60):
        """
        Prepares a dataset of assembly graphs for GAT model training
        Args:
            graph_list: List of NetworkX digraphs representing assembly code
            tokenizer: HierarchicalAssemblyTokenizer instance
            node_feature_dim: Dimension of node embeddings
        """
        self.graph_list = graph_list
        self.tokenizer = tokenizer
        self.node_feature_dim = node_feature_dim

        # Initialize embedding layers for each feature type
        num_feature_type = 5
        vocab_sizes = tokenizer.get_vocab_sizes()
        self.node_type_embedding = nn.Embedding(vocab_sizes['node_type'], node_feature_dim // num_feature_type)
        self.operation_embedding = nn.Embedding(vocab_sizes['operation'], node_feature_dim // num_feature_type)
        self.register_embedding = nn.Embedding(vocab_sizes['register'], node_feature_dim // num_feature_type)
        self.memory_pattern_embedding = nn.Embedding(vocab_sizes['memory_pattern'], node_feature_dim // num_feature_type)
        self.immediate_embedding = nn.Embedding(vocab_sizes['immediate'], node_feature_dim // num_feature_type)

        # Process graphs into PyTorch Geometric Data objects
        self.data_list = []
        for graph in self.graph_list:
            # process graph info
            self.data_list.append(self._process_graph(graph['graph_input'],graph['name']))

    def _process_graph(self, graph, hash_name):
        """Convert a NetworkX graph to a PyTorch Geometric Data object"""
        # Create a node ID mapping for consecutive IDs
        node_mapping = {node: i for i, node in enumerate(graph.nodes())}

        # Get node features
        node_features_dict = self.tokenizer.encode_graph(graph)

        # Convert to tensor-friendly format
        x = torch.zeros((len(graph.nodes()), 7), dtype=torch.long)
        for node_id, features in node_features_dict.items():
            x[node_mapping[node_id]] = torch.tensor(features)

        # Create edge index
        edge_list = list(graph.edges())
        if not edge_list:
            # Handle case with no edges
            edge_index = torch.zeros((2, 0), dtype=torch.long)
        else:
            edge_index = self.optimize_edge_index(edge_list, node_mapping)

        # Endcode Hash name
        encoded_hash = self.tokenizer.encode_hash(hash_name)
        encoded_hash_tensor = torch.tensor(encoded_hash, dtype=torch.long)

        # Get embeddings before creating Data object
        x = self.get_embeddings(x)

        # Create Data object
        data = Data(x=x, edge_index=edge_index, hash_encoded=encoded_hash_tensor)
        return data

    def optimize_edge_index(self, edge_list, node_mapping):
        """
        Optimizes the creation of edge_index tensor for graph representation.
        Args:
            edge_list (list of tuples): List of edge tuples (src, tgt).
            node_mapping (dict): Dictionary mapping node IDs to integer indices.
        Returns:
            torch.Tensor: Optimized edge_index tensor.
        """
        if not edge_list:
            return torch.zeros((2, 0), dtype=torch.long)

        num_edges = len(edge_list)
        src_indices = torch.empty(num_edges, dtype=torch.long)
        tgt_indices = torch.empty(num_edges, dtype=torch.long)

        for i, (src, tgt) in enumerate(edge_list):
            src_indices[i] = node_mapping[src]
            tgt_indices[i] = node_mapping[tgt]

        return torch.stack((src_indices, tgt_indices), dim=0).contiguous()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

    def get_embeddings(self, x):
        """Transform tokenized features into embeddings"""
        # Split features into their components
        node_type_idx = x[:, 0]
        operation_idx = x[:, 1]
        register_idx1 = x[:, 2]
        register_idx2 = x[:, 3]
        register_idx3 = x[:, 4]
        memory_pattern_idx = x[:, 5]
        immediate_idx = x[:, 6]

        # Get embeddings for each component
        node_type_emb = self.node_type_embedding(node_type_idx)
        operation_emb = self.operation_embedding(operation_idx)

        # Combine register embeddings (average them)
        register_emb = (self.register_embedding(register_idx1) +
                        self.register_embedding(register_idx2) +
                        self.register_embedding(register_idx3)) / 3

        memory_pattern_emb = self.memory_pattern_embedding(memory_pattern_idx)
        immediate_emb = self.immediate_embedding(immediate_idx)

        # Concatenate all embeddings
        return torch.cat([
            node_type_emb,
            operation_emb,
            register_emb,
            memory_pattern_emb,
            immediate_emb
        ], dim=1)

"""
Graph Attention Network for assembly code analysis
Args:
    dataset: AssemblyGraphDataset instance
    hidden_dim: Hidden dimension of GAT layers
    output_dim: Output dimension of node embeddings
    heads: Number of attention heads
    dropout: Dropout rate
"""
class AssemblyGAT(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim=64, output_dim=453, heads=8, dropout=0.6, hash_dim=512):
        super(AssemblyGAT, self).__init__()
        self.conv1 = GATConv(node_feature_dim, hidden_dim, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout
        self.hash_linear = nn.Linear(hash_dim, node_feature_dim) # Linear layer for hash encoding, adjusted dim
        self.hash_dim = hash_dim

    def forward(self, x, edge_index, hash_encoded, batch):
        """Forward pass through the GAT model"""
        # First GAT layer with activation and dropout
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Second GAT layer
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

# Function to parse digraph string (as in previous example)
def parse_digraph_string(digraph_str):
    """Parse the digraph string to create a NetworkX DiGraph"""
    # Simple parser for the provided format
    G = nx.DiGraph()

    # Regular expressions for node and edge definitions
    node_pattern = re.compile(r'"([^"]+)"\s*\[label\s*=\s*"([^"]+)"\]')
    edge_pattern = re.compile(r'"([^"]+)"\s*->\s*"([^"]+)"')

    # Extract nodes and edges
    for line in digraph_str.strip().split('\n'):
        line = line.strip()

        # Skip the Digraph G { and } lines
        if line == 'Digraph G {' or line == '}':
            continue

        # Parse node definitions
        node_match = node_pattern.search(line)
        if node_match:
            node_id = node_match.group(1)
            label = node_match.group(2)
            G.add_node(node_id, label=label)
            continue

        # Parse edge definitions
        edge_match = edge_pattern.search(line)
        if edge_match:
            source = edge_match.group(1)
            target = edge_match.group(2)
            G.add_edge(source, target)
    return G

In [12]:
# Create empty list to store dataframes
# init parameters to loop the file loading by batch
files_meta_list = set(df_meta_train['name'].astype(str))
random_seed = 42

def process_batch(file_batch):
    """
    Processes a batch of files.
    Args:
        file_batch (list): A list of file paths.
    """
    graph_list_curr = []
    file_with_err = 0

    for full_path_file in file_batch:
        try:
            with open(full_path_file, 'r') as f:
                hash_file = full_path_file.split('.jso')[0]
                hash_file = hash_file.split(split_char)[-1]

                #f = open(full_path_file, 'r')
                digraph_str = f.read()

                # test if file content has no error tag
                if 'ERROR' in digraph_str:
                    file_with_err += 1
                    continue

                G = parse_digraph_string(digraph_str)
                graph_list_curr.append({
                    'name': hash_file,
                    'graph_input' : G
                })
                f.close()

        except FileNotFoundError:
            print(f"Error: File not found: {full_path_file}")
        except Exception as e:
            print(f"Error processing {full_path_file}: {e}")

    return graph_list_curr

def get_metadata_from_hash(hash_list):
    hash_meta_values = []
    for hash_name in hash_list:
        df_y_values = df_meta_train[df_meta_train['name'] == hash_name].copy()
        df_y_values.drop(columns=['name'], inplace=True)
        if len(df_y_values) > 0 :
             hash_meta_values.append(df_y_values.values[0]) # Get the first row's values
        else:
            print(f"{hash_name} not found in metadata")
    return hash_meta_values

def train_model(train_dataloader, train_hash, batch_size, epochs):
    if os.path.exists(model_save_file):
      if torch.cuda.is_available():
          model.load_state_dict(torch.load(model_save_file, weights_only=True))
      else:
          model.load_state_dict(torch.load(model_save_file, weights_only=True, map_location=torch.device('cpu')))

    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        count_batch = 0

        for batch in train_dataloader:
            optimizer.zero_grad()  # Zero gradients

            # get the different components of the dataset
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            hash_encoded = batch.hash_encoded.to(device)
            #print(f"batch: {count_batch}  == x: {x.shape} edge_index: {edge_index.shape} ")
            list_hash_batch = train_hash[count_batch*batch_size:(count_batch+1)*batch_size]
            y_label = get_metadata_from_hash(list_hash_batch)
            y_label = np.array(y_label)
            y_label = torch.tensor(y_label, dtype=torch.float32).to(device)

            bincount_list = torch.bincount(batch.batch).tolist()
            if any(bincount_list): #checks if any values are not zero.
                batch_p = torch.cat([torch.full((num_nodes_graph_i,), i, device=device) for i, num_nodes_graph_i in enumerate(bincount_list)])
                batch_p = batch_p.to(device)
            else:
                # Handle the case where the list is empty. For example, create an empty tensor or raise an error.
                print("Warning: bincount_list is empty. Creating an empty tensor.")
                batch_p = torch.empty(0, dtype=torch.long, device=device) # create empty tensor.

            # Forward pass
            output = model(x, edge_index, hash_encoded, batch_p)
            count_batch += 1

            if output.shape != y_label.shape:
                print(f"Warning: output shape {output.shape} and y_label shape {y_label.shape} do not match.")
                break

            loss = F.binary_cross_entropy_with_logits(output, y_label.float())
            train_loss += loss.item()

            loss.backward(retain_graph=True)
            optimizer.step()

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        #avg_train_loss = train_loss / len(train_dataloader)
        #print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # save model trained
    torch.save(model.state_dict(), model_save_file)

def validate_model_perf(validation_dataloader, val_hash, batch_size):
    if os.path.exists(model_save_file):
        model.load_state_dict(torch.load(model_save_file, weights_only=True))

    model.eval() #set the model to evaluation mode.
    all_true_label = []
    all_pred_label = []
    #hash_v = []

    count_batch = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            hash_encoded = batch.hash_encoded.to(device)

            list_hash_batch = val_hash[count_batch*batch_size:(count_batch+1)*batch_size]
            y_label = get_metadata_from_hash(list_hash_batch)
            y_label = np.array(y_label)
            y_label = torch.tensor(y_label, dtype=torch.float32).to(device)

            bincount_list = torch.bincount(batch.batch).tolist()
            if any(bincount_list): #checks if any values are not zero.
                batch_p = torch.cat([torch.full((num_nodes_graph_i,), i, device=device) for i, num_nodes_graph_i in enumerate(bincount_list)])
                batch_p = batch_p.to(device)
            else:
                # Handle the case where the list is empty. For example, create an empty tensor or raise an error.
                print("Warning: bincount_list is empty. Creating an empty tensor.")
                batch_p = torch.empty(0, dtype=torch.long, device=device) # create empty tensor.

            # Forward pass
            output = model(x, edge_index, hash_encoded, batch_p)
            count_batch += 1

            if output.shape != y_label.shape:
                print(f"Warning: output shape {output.shape} and y_label shape {y_label.shape} do not match.")
                break

            predicted_labels_prob = torch.sigmoid(output)
            prediction_labels = (predicted_labels_prob > 0.5).int()
            all_true_label.append(y_label.cpu())
            all_pred_label.append(prediction_labels.cpu())
            #hash_v.append(list_hash_batch)

    true_labels_tensor = torch.cat(all_true_label, dim=0) # Concatenate along dimension 0
    pred_labels_tensor = torch.cat(all_pred_label, dim=0) # Concatenate along dimension 0

    return pred_labels_tensor, true_labels_tensor

def run_files_in_batches(full_file_paths, batch_files_size=50, mode='train'):
    """
    Processes files in batches for tokenization and model training/evaluation in a single loop.
    Args:
        full_file_paths (list): List of full file paths to process.
        batch_files_size (int): Number of files to process in each batch for tokenization.
        batch_data_size (int): Batch size for the DataLoader during training/evaluation.
        num_feature_dim (int, optional): Dimension of node features. Defaults to None.
        mode (str): 'train' or other mode (e.g., 'eval'). Defaults to 'train'.
        epochs (int): Number of training epochs per batch (if mode is 'train'). Defaults to 3.
        tokenizer: Tokenizer object with methods like get_tokens_and_counts and fit_from_counts.
        process_batch (callable): Function to process a batch of file paths and return a list of graph-like objects.
        train_model (callable): Function to train the model with a DataLoader and other info.
    """
    print(f"There are {len(full_file_paths)} files for {mode}")

    num_files = len(full_file_paths)
    nb_batch = (num_files // batch_files_size) + 1

    hash_v = []
    hash_t = []
    all_true_label = []
    all_pred_label = []

    all_counts = {
        'node_types': [],
        'operations': [],
        'registers': [],
        'memory_patterns': [],
        'immediates': [],
        'unique_hashes': [],
    }

    print("**** Tokenization and Dataset Processing & Training")
    for i in range(0, num_files, batch_files_size):
        batch_files = full_file_paths[i:i + batch_files_size]
        batch_num = i // batch_files_size + 1
        print(f"Processing batch {batch_num}/{nb_batch}: {len(batch_files)} files")

        batch_graph_list = process_batch(batch_files)
        if not batch_graph_list:
            print("graph list is empty for this batch")
            continue

        # Tokenization for the current batch
        tokens = tokenizer.get_tokens_and_counts(batch_graph_list)
        for key in all_counts:
            if key in tokens:
                all_counts[key].extend(tokens[key])
        tokenizer.fit_from_counts(all_counts)
        # Create Dataset and DataLoader for the current batch
        if tokenizer and num_feature_dim is not None:
            dataset = AssemblyGraphDataset(batch_graph_list, tokenizer, node_feature_dim=num_feature_dim)
            curr_dataloader = DataLoader(dataset, batch_size=batch_data_size, shuffle=False)
            list_of_hash = [graph['name'] for graph in batch_graph_list]

            # Training or other processing for the current batch
            if mode == 'train' and train_model:
                print(f"**** Training on batch {batch_num}/{nb_batch}")
                train_model(curr_dataloader, list_of_hash, batch_data_size, epochs=epochs)
                hash_t.extend(list_of_hash)
            # Add logic for other modes (e.g., 'eval') here if needed
            else:
                pred_labels, true_labels = validate_model_perf(curr_dataloader, list_of_hash, batch_data_size)
                all_true_label.append(true_labels.cpu())
                all_pred_label.append(pred_labels.cpu())
                hash_v.append(list_of_hash)

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Final fitting of the tokenizer after processing all batches
    if all_counts:
        print("**** Finalizing Tokenizer Vocabulary")
        tokenizer.fit_from_counts(all_counts)

    #print("**** Processing Complete")
    # save all labels as tensor to return if mode= 'validation'
    if mode == 'validation':
        all_true_label_tensor = torch.cat(all_true_label).cpu().numpy()
        all_pred_label_tensor = torch.cat(all_pred_label).cpu().numpy()

        return all_pred_label_tensor, all_true_label_tensor, hash_v
    elif mode == 'train':
        return hash_t


In [13]:
###############################################################
# Main init and training sequence
###############################################################
# Batch params
batch_data_size = 1
batch_files_size = 1
num_feature_dim = 15
epochs =  5

# Initialize model
tokenizer = HierarchicalAssemblyTokenizer()
model = AssemblyGAT(node_feature_dim=num_feature_dim, hidden_dim=64, output_dim=453)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

batch_curr = 'rep_0'
for i in range(0, 100, 500):
  batch_curr = f'rep_{i}'
  print(f"========== batch: {batch_curr} ========== ")
  list_train, list_val, list_err = process_graph_directory(training_path_dir, val_size=0.01, rep_batch=batch_curr, max_file=0)

  # validate
  all_pred_label_tensor, all_true_label_tensor, hash_v = run_files_in_batches(list_val, batch_files_size=batch_files_size, mode='validation')
  f1_macro = f1_score(all_true_label_tensor, all_pred_label_tensor, average='macro', zero_division=1)
  print(f"**** F1 score: {f1_macro*100:.2f} \n")



========== batch: rep_0 ========== 
Number of files with error: 0
There are 81 files for validation
**** Tokenization and Dataset Processing & Training
Processing batch 1/82: 1 files
Processing batch 2/82: 1 files
Processing batch 3/82: 1 files
Processing batch 4/82: 1 files
Processing batch 5/82: 1 files
Processing batch 6/82: 1 files
Processing batch 7/82: 1 files
Processing batch 8/82: 1 files
Processing batch 9/82: 1 files
Processing batch 10/82: 1 files
Processing batch 11/82: 1 files
Processing batch 12/82: 1 files
Processing batch 13/82: 1 files
Processing batch 14/82: 1 files
Processing batch 15/82: 1 files
Processing batch 16/82: 1 files
graph list is empty for this batch
Processing batch 17/82: 1 files
Processing batch 18/82: 1 files
Processing batch 19/82: 1 files
Processing batch 20/82: 1 files
Processing batch 21/82: 1 files
Processing batch 22/82: 1 files
Processing batch 23/82: 1 files
Processing batch 24/82: 1 files
Processing batch 25/82: 1 files
Processing batch 26/82

In [14]:
df_result = pd.read_csv(r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\Pred_test_v12_500_2803\test_prediction_val.csv")
df_res_pred = df_result
df_res_true = df_result
df_result.head()

,name,type,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,...,winzip,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda


In [15]:
  #all_pred_label_tensor, all_true_label_tensor, hash_v


for row_index in range(len(all_pred_label_tensor)):
  print(list_val[row_index])
  pred_list = all_pred_label_tensor[row_index].tolist()  # Convert tensor row to list
  int_pred_list = [int(val) for val in pred_list]  # Convert to int
  df_res_pred.loc[row_index,0]=list_val[row_index]

  true_list = all_true_label_tensor[row_index].tolist()
  int_true_list = [int(val) for val in true_list]  # Convert to int
  df_res_true.loc[row_index, 0]=list_val[row_index]
  # Update the DataFrame
  for j in range(len(int_true_list)):
      df_res_pred.iloc[row_index, j+2] = int_pred_list[j]
      df_res_true.iloc[row_index, j+2] = int_true_list[j]



G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\40936a67037f8fd8e215b045f9cdf9c55840411316a62c85c8b54f75c6b0a5c8.json
G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\368d2b5ccfd49f01942f462037710146ec3ca5ca8a5318a092bc49bfebfe8bad.json
G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\75010f6d21b0b4451b5465ab8f46b385bb1edc15ed50634f4120352edc49cf3a.json
G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\95e944577e686782e9d725e2e79ef9657a3f3b4f3b9cb265487bcd8e55cba95d.json
G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\72356f26be98c580e23ca7baebe665c781fb2484575b9b05ad38676ad74ffb9e.json
G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\76b1b2485d06f974b85c

In [16]:
df_res_pred.head()

,name,type,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,...,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda,0
0,NaN,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1...
1,NaN,NaN,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1...
2,NaN,NaN,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1...
3,NaN,NaN,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1...
4,NaN,NaN,0,0,0,0,0,1,0,0,...,0,0,1,1,0,1,0,0,0,G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1...


In [17]:
# on sauve les results partiels
df_res_true.to_csv(f"val_true.csv", header=True ,index=False )
df_res_pred.to_csv(f"val_pred.csv", header=True ,index=False )

In [18]:
import shap


# --- SHAP Analysis ---

# Explainer pour le modèle linéaire
explainer = shap.LinearExplainer(model, X_train)

# Calculer les valeurs SHAP pour l'ensemble de test
shap_values = explainer.shap_values(X_test)

# Visualisation du résumé des valeurs SHAP
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.title("Résumé de l'Importance des Caractéristiques (via SHAP)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test)
plt.title("Impact des Caractéristiques sur la Prédiction (via SHAP)")
plt.tight_layout()
plt.show()

# Analyse de la dépendance SHAP pour l'éducation
plt.figure(figsize=(8, 6))
shap.dependence_plot("Education", shap_values, X_test)
plt.title("Dépendance SHAP pour l'Éducation")
plt.tight_layout()
plt.show()

# --- SHAP Analysis avec le facteur de confusion inclus ---

X_with_S = synthetic_data[['Education', 'Experience', 'City_Size', 'Languages_Spoken', 'Socioeconomic_Status']]
Y_salary_with_S = synthetic_data['Salary']
X_train_S, X_test_S, Y_train_S, Y_test_S = train_test_split(X_with_S, Y_salary_with_S, test_size=0.2, random_state=42)

model_with_S = LinearRegression()
model_with_S.fit(X_train_S, Y_train_S)

explainer_with_S = shap.LinearExplainer(model_with_S, X_train_S)
shap_values_with_S = explainer_with_S.shap_values(X_test_S)

plt.figure(figsize=(12, 7))
shap.summary_plot(shap_values_with_S, X_test_S, plot_type="bar")
plt.title("Résumé de l'Importance des Caractéristiques (avec Statut Socioéconomique)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 7))
shap.summary_plot(shap_values_with_S, X_test_S)
plt.title("Impact des Caractéristiques sur la Prédiction (avec Statut Socioéconomique)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
shap.dependence_plot("Education", shap_values_with_S, X_test_S)
plt.title("Dépendance SHAP pour l'Éducation (avec Statut Socioéconomique)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
shap.dependence_plot("Socioeconomic_Status", shap_values_with_S, X_test_S)
plt.title("Dépendance SHAP pour le Statut Socioéconomique")
plt.tight_layout()
plt.show()

print("\nExplication avec SHAP (SHapley Additive exPlanations):")
print("----------------------------------------------------")
print("SHAP values provide insights into how each feature contributes to the prediction of each individual instance.")
print("They decompose the prediction by quantifying the additive contribution of each feature.")

print("\n**Analyse du premier modèle (sans Statut Socioéconomique):**")
print("- Le **Summary Plot (bar)** montre l'importance globale de chaque caractéristique pour le modèle. Les barres plus longues indiquent une plus grande importance.")
print("- Le **Summary Plot (points)** donne plus d'informations :")
print("  - L'axe horizontal représente la valeur SHAP (l'impact de la caractéristique sur la prédiction).")
print("  - L'axe vertical liste les caractéristiques, ordonnées par importance.")
print("  - Chaque point représente une instance du jeu de données.")
print("  - La couleur du point indique la valeur de la caractéristique (rouge = haute, bleu = basse).")
print("  - On peut observer si une valeur élevée d'une caractéristique tend à augmenter ou diminuer la prédiction.")
print("- Le **Dependence Plot** pour l'Éducation montre comment la valeur SHAP de l'Éducation varie en fonction de sa valeur réelle. Il peut révéler des relations non linéaires et les interactions avec d'autres caractéristiques (la couleur des points peut représenter une autre caractéristique).")

print("\n**Analyse du deuxième modèle (avec Statut Socioéconomique):**")
print("- En incluant le 'Socioeconomic_Status' dans le modèle, les valeurs SHAP pour les autres caractéristiques (comme l'Éducation) peuvent changer.")
print("- Le nouveau **Summary Plot** montrera l'importance relative de toutes les caractéristiques, y compris le 'Socioeconomic_Status'. On s'attend à ce que le 'Socioeconomic_Status' ait une importance significative étant donné son rôle de confondeur dans la génération des données.")
print("- Le **Dependence Plot** pour l'Éducation dans ce deuxième modèle devrait montrer l'impact de l'Éducation sur le salaire *après avoir tenu compte* du 'Socioeconomic_Status'. La relation observée pourrait être différente de celle du premier modèle.")
print("- Le **Dependence Plot** pour le 'Socioeconomic_Status' montrera son impact direct sur la prédiction du salaire dans ce modèle.")

print("\n**Interprétation concernant le biais de confusion avec SHAP:**")
print("- En comparant les valeurs SHAP et les summary plots des deux modèles, on peut observer comment l'importance et l'impact des autres caractéristiques (notamment l'Éducation et potentiellement l'Expérience) changent lorsque le facteur de confusion est inclus dans le modèle.")
print("- Si le biais de confusion est présent, le premier modèle pourrait surestimer ou sous-estimer l'impact direct de certaines caractéristiques sur le salaire, car il capture également l'effet indirect via le confondeur.")
print("- Le deuxième modèle, en incluant le confondeur, fournit des estimations SHAP qui reflètent l'impact direct de chaque caractéristique *conditionnellement* aux autres caractéristiques, y compris le confondeur, aidant ainsi à démêler les relations causales potentielles.")


ModuleNotFoundError: No module named 'shap'